In [9]:
#Make_DereivedStagingTables.ipynb
##last modified 20230309 

#You will need to manually amend the target dataset for this script
#The target dataset is a termporary location into which you build objects that will then be used to build the fdm


library(bigrquery)
library(lubridate) 

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

# This script uses the primary care FDM  and the warehouse FDM for its data .

sourcedb1 <-"yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8"

# This can be a number of datasets for this example I am just using the warehouse
sourcedb2 <-"yhcr-prd-phm-bia-core.CB_FDM_Warehouse_V3"
#sourcedb3 <-"yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_SUS_Airedale_FDM_Format"

targetdb <-"yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"

print (sourcedb1) 
print (sourcedb2) 
print (targetdb)
#print(lookups)



[1] "yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8"
[1] "yhcr-prd-phm-bia-core.CB_FDM_Warehouse_V3"
[1] "yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"


In [10]:
# So I want to build teh cohort from Pirimayr Care , so I need that bit done first 
# I'm going to use the primary care.tbl_srcode table and the temp table we loaded 
# in this case it is cardio - but it could be any other code list

# first I'm going to build a temp table for teh cohot person_id


sql4 <-paste('drop table if exists ',targetdb,'.tmp_cohort;', sep = "")
tb4 <-bq_project_query(project_id, sql4)

sql5 <-paste(' create table ',targetdb,'.tmp_cohort
as 
select distinct person_id  from ',sourcedb1,'.tbl_srcode cc
WHERE CTV3Code IN (select Code from ',targetdb,'.tmp_cardio)
and person_id in (
select person_id 
 from CB_FDM_MASTER.person 
 where DATE_DIFF(current_date(), birth_datetime, YEAR) >64
 and death_datetime is null) ', sep = "")
tb5 <-bq_project_query(project_id, sql5)

In [11]:
# Now I'll copy tbl_Srcode into myspace

sql4 <-paste('drop table if exists ',targetdb,'.tbl_srcode;', sep = "")
tb4 <-bq_project_query(project_id, sql4)


sql5 <-paste(' create table ',targetdb,'.tbl_srcode 
as 
select cc.* 
from ',sourcedb1,'.tbl_srcode cc ,
',targetdb,'.tmp_cohort co 
where cc.person_id = co.person_id', sep = "")
tb5 <-bq_project_query(project_id, sql5)


# This copies the tbl_srcode into our space but only for those patients
# in the cardio table
# and ages over 64 

In [12]:
# This section is where I copy in the other source tables (again just for teh demographic above 
# using the tmp_cohort table I created earier

sql5 <-paste('drop table if exists ',targetdb,'.tbl_ae_nautilus;', sep = "")
tb5 <-bq_project_query(project_id, sql5)

sql8 <-paste('create or replace TABLE ', targetdb , '.tbl_ae_nautilus
as 
SELECT 
src.* FROM ',sourcedb2,'.tbl_ae_nautilus src ,
',targetdb,'.tmp_cohort co 
where src.person_id = co.person_id', sep = "")

tb8 <-bq_project_query(project_id, sql8)

In [13]:

# Now I just repeat - adding in any source tables (for that cohort) to my space
# preparing it to be FDM'd

sql6 <-paste('drop table if exists ',targetdb,'.tbl_apc_finished_susplus;', sep = "")
tb6 <-bq_project_query(project_id, sql6)


sql9<-paste('create or replace TABLE ', targetdb , '.tbl_apc_finished_susplus 
as 
Select 
src.* FROM ', sourcedb2,'.tbl_apc_finished_susplus  src ,
',targetdb,'.tmp_cohort co 
where src.person_id = co.person_id', sep = "")

tb9 <-bq_project_query(project_id, sql9)

In [ ]:
# ADD IN NEW TABLES AS REQUIRED.